
# 01. 대용량 데이터 샘플링 전략 (Data Sampling)

본 노트북은 8GB 이상의 대용량 Parquet 데이터를 효율적으로 처리하기 위한 **샘플링 전략**을 다룹니다.
전체 데이터를 메모리에 로드하지 않고, `pyarrow`의 Row Group 기능을 활용하여 메모리 효율적으로 부분 데이터를 추출하는 방법을 기술합니다.

### 📋 주요 프로세스
1. **Row Group 기반 읽기**: 전체 파일을 읽지 않고 메타데이터만 로드
2. **Systematic Sampling**: 전체 데이터 분포를 유지하기 위해 1/10 비율로 등간격 추출
3. **Parquet 저장**: 분석용 경량 데이터셋(`train_sample_10pct.parquet`) 생성


In [1]:
import pandas as pd
import pyarrow.parquet as pq
import os

# 파일 경로 설정
base_path = r"d:\folder\대학원\CTR\rawdata"
source_file = os.path.join(base_path, "train.parquet")
save_file = os.path.join(base_path, "train_sampled_10pct.parquet")

print(f"원본 파일 경로: {source_file}")
print(f"저장될 파일 경로: {save_file}")

원본 파일 경로: d:\folder\대학원\CTR\rawdata\train.parquet
저장될 파일 경로: d:\folder\대학원\CTR\rawdata\train_sampled_10pct.parquet


In [ ]:
def create_sampled_dataset(source_path, save_path, sample_ratio=10):
    """
    대용량 Parquet 데이터를 읽어 약 1/n로 샘플링하여 저장하는 함수
    """
    if not os.path.exists(source_path):
        print("❌ 원본 파일을 찾을 수 없습니다.")
        return

    print(f"🔄 데이터 로드 및 샘플링 시작... (1/{sample_ratio} 크기로 줄임)")
    
    # 1. Parquet 파일 메타데이터 읽기
    parquet_file = pq.ParquetFile(source_path)
    num_row_groups = parquet_file.num_row_groups
    
    print(f"📊 전체 Row Group 수: {num_row_groups}")
    
    # 2. Row Group 단위로 건너뛰며 읽기 (메모리 효율적 방법)
    # 0, 10, 20... 인덱스의 그룹만 읽음
    groups_to_read = list(range(0, num_row_groups, sample_ratio))
    
    print(f"✂️ {sample_ratio}개 그룹마다 1개씩 추출하여 총 {len(groups_to_read)}개 그룹을 로드합니다.")
    
    # 선택된 그룹만 읽어서 DataFrame으로 변환
    df_sampled = parquet_file.read_row_groups(groups_to_read).to_pandas()
    
    print(f"✅ 샘플링 완료! 데이터 크기: {df_sampled.shape}")
    
    # 3. 샘플링된 데이터 저장
    print(f"💾 파일을 저장하는 중...")
    df_sampled.to_parquet(save_path, index=False)
    print(f"🎉 저장 완료!\n파일 위치: {save_path}")

: 

In [ ]:
# 함수 실행
create_sampled_dataset(source_file, save_file, sample_ratio=10)

🔄 데이터 로드 및 샘플링 시작... (1/10 크기로 줄임)
📊 전체 Row Group 수: 1
✂️ 10개 그룹마다 1개씩 추출하여 총 1개 그룹을 로드합니다.


In [ ]:
# 결과 확인: 잘 만들어졌는지 로드해서 상위 5개만 보기
if os.path.exists(save_file):
    df_check = pd.read_parquet(save_file)
    print("데이터 샘플 미리보기:")
    display(df_check.head())

NameError: name 'os' is not defined